In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

**Предварительная обработка**

In [2]:
import math

def bilinear_kernel(src, x, y):
    x1 = int(math.floor(x))
    y1 = int(math.floor(y))
    x2 = min(x1 + 1, src.shape[1] - 1)
    y2 = min(y1 + 1, src.shape[0] - 1)

    i_11 = src[y1, x1, :]
    i_12 = src[y2, x1, :]
    i_21 = src[y1, x2, :]
    i_22 = src[y2, x2, :]

    return i_11 * (x2 - x) * (y2 - y) + \
           i_12 * (x2 - x) * (y - y1) + \
           i_21 * (x - x1) * (y2 - y) + \
           i_22 * (x - x1) * (y - y1)

In [3]:
i = cv2.imread("./img.png")
i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
img = i
plt.imshow(i)
cv2.imwrite("out.png", i)

True

In [4]:
# Параметры центральной проекции
f_x = 943.37 # пиксельное фокусное расстояние по оси X
f_y = 943.37 # пиксельное фокусное расстояние по оси Y
u_x = 990.38 # координаты центрального пикселя (principal point) по X 
u_y = 487.67 # координаты центрального пикселя (principal point) по Y

# Матрица центральной проекции
K = np.array(
    [
        [f_x, 0.0, u_x],
        [0.0, f_y, u_y],
        [0.0, 0.0, 1.0]
    ],
    dtype=np.float32 # opencv работает с float32
)

# Параметры радиальной дисторсии
k_1 = -0.3229
k_2 = 0.1042
k_3 = -0.0148

D = np.array([k_1, k_2, 0.0, 0.0, k_3], dtype=np.float32)

# Размеры изображения
image_size = (1920, 1080)

# Углы поворота с.к. камеры
r_1 = -0.6344
r_2 = -0.0416
r_3 = 0.0157

# координаты установки камеры в с.к. робота
t_1 = -0.378
t_2 = -3.915
t_3 = 2.400

print('K = ', K)
print('D = ', D)
print('image_size = ', image_size)

K =  [[943.37   0.   990.38]
 [  0.   943.37 487.67]
 [  0.     0.     1.  ]]
D =  [-0.3229  0.1042  0.      0.     -0.0148]
image_size =  (1920, 1080)


In [5]:
import numpy.linalg
# матрица поворота
R = np.linalg.inv(cv2.Rodrigues(np.array([r_1, r_2, r_3], dtype=np.float32))[0])

t = np.array([[t_1], [t_2], [t_3]], dtype=np.float32)
RT = np.hstack((R, -R@t))
print(f"R|t = \n{RT}")

# матрица камеры наблюдения
P = K @ RT
print(f"P = \n{P}")

def proj_pt(P, r):
    h_pix = P @ np.hstack((r, [1]))
    return (h_pix[0] / h_pix[2], h_pix[1] / h_pix[2])

print(proj_pt(P, [0,0,5]))
print(proj_pt(P, [0,0,6]))
print(proj_pt(P, [0,0,7]))
print(proj_pt(P, [3,0,10]))
print(proj_pt(P, [3,0,20]))

R|t = 
[[ 9.9904436e-01  2.7419649e-02  3.4037601e-02  4.0329644e-01]
 [-1.9061693e-03  8.0534059e-01 -5.9280938e-01  4.5749302e+00]
 [-4.3666486e-02  5.9217793e-01  8.0462325e-01  3.7077475e-01]]
P = 
[[ 8.9922205e+02  6.1234808e+02  8.2899280e+02  7.4766565e+02]
 [-2.3093060e+01  1.0485216e+03 -1.6684796e+02  4.4966680e+03]
 [-4.3666486e-02  5.9217793e-01  8.0462325e-01  3.7077475e-01]]
(1113.5072910064018, 833.5273180128322)
(1100.626484986871, 672.4190894127535)
(1091.1986053765831, 554.4987526601302)
(1416.2742925893558, 332.96000424490273)
(1226.1139613020323, 66.76546189704504)


In [6]:
# уравнение плоскости проекции
plane = np.array([0,1,0,0], dtype=np.float32) # Y = 0

# применим формулы для вычисления гомографии с предыдущей лекции
P_plane = np.vstack((P, plane))
H = P @ np.linalg.inv(P_plane)[:, :3]
rH = np.linalg.inv(K).dot(H).dot(K)

print(H)

[[ 1.0000000e+00  4.4411674e-09 -4.2468650e-06]
 [ 7.6142292e-10  1.0000000e+00  6.4000924e-06]
 [-1.9657689e-12 -2.6244215e-12  9.9999994e-01]]


## Задача 1
Напишите класс для вычисления карты пересчета undistort-преобразования.
Сравните результат с выводом функции cv2.initUndistortRectifyMap

In [7]:
# вывод функции cv2.initUnistortRectifyMap
rmap_x, rmap_y = cv2.initUndistortRectifyMap(K, D, rH, K, image_size, cv2.CV_32FC1)
lib_undistort = cv2.remap(i, rmap_x, rmap_y, interpolation=cv2.INTER_LINEAR)

cv2.imwrite("lib_undistort.png", lib_undistort)

True

![undistirt-преобразование с помощью cv2](./lib_undistort.png)

In [8]:
class UndistortRemap:
    def __init__(self, image_size, K, D, rH):
        self.image_size = image_size
        self.K = K
        self.D = D
        self.rH = rH
        
    def r2(self, x, y):
        return (x**2 + y**2)
    
    def coef(self, x, y):
        r2_ = self.r2(x, y)
        r4_ = r2_ ** 2
        r6_ = r4_ * r2_
        return (1 + self.D[0] * r2_ + self.D[1] * r4_ + self.D[4] * r6_)
        
    def normalize(self, x, y):
        return ((x - self.K[0][2]) / self.K[0][0], (y - self.K[1][2]) / self.K[1][1])
    
    def unnormalize(self, x, y):
        return (x * self.K[0][0] + self.K[0][2], y * self.K[1][1] + self.K[1][2])
    
    def proj(self, x, y):
        pt_homo = np.linalg.inv(self.rH).dot(np.array([x, y, 1], dtype=np.float32).T)
        return (pt_homo[0] / pt_homo[2], pt_homo[1] / pt_homo[2])

    def undistort(self, x, y):
        x_norm, y_norm = self.normalize(x, y)
        xx, yy = self.proj(x_norm, y_norm)
        x_c = xx * self.coef(xx, yy)
        y_c = yy * self.coef(xx, yy)
#         xx_1, yy_1, a = np.array([[self.rH[2][2], 0, -self.rH[0][2]], [0, self.rH[2][2], -self.rH[1][2]], [0, 0, 1]], dtype=np.float32).dot(self.rH).dot(np.array([x_c, y_c, 1], dtype=np.float32))
        x_res, y_res = self.unnormalize(x_c, y_c)
        return (x_res, y_res)

In [9]:
remapper = UndistortRemap(image_size, K, D, rH)

rmap_x = np.zeros(shape=(image_size[1], image_size[0]), dtype=np.float32)
rmap_y = np.zeros(shape=(image_size[1], image_size[0]), dtype=np.float32)

width = image_size[0]
height = image_size[1]

for x in np.arange(0, width):
    for y in np.arange(0, height):
        x_res, y_res = remapper.undistort(x, y)
        if x_res >= 0 and x_res < width:
            rmap_x[y, x] = x_res
        if y_res >= 0 and y_res < height:
            rmap_y[y, x] = y_res

In [10]:
my_undistort = cv2.remap(i, rmap_x, rmap_y, interpolation=cv2.INTER_LINEAR)

cv2.imwrite("myundistort.png", my_undistort)

True

![undistort-преобразование](./myundistort.png)

## Задача 2
Вычислить карту пересчета для преобразования undistort в определенной области исходного изображения (ROI + Undistort)

In [11]:
roi_x = 100
roi_y = 600
roi_w = 1000
roi_h = 400

In [12]:
class RoiUndistortRemap:
    def __init__(self, x, y, w, h, undistort_remapper):
        self.x0 = x
        self.y0 = y
        self.w = w
        self.h = h
        self.undistort_remapper = undistort_remapper
    
    def x(self, x, y):
        if x >= self.w or y >= self.h:
            return 0.0
        return self.undistort_remapper.undistort(x+self.x0,y)[0]

    def y(self, x, y):
        if x >= self.w or y >= self.h:
            return 0.0
        return self.undistort_remapper.undistort(x,y+self.y0)[1]

In [13]:
remapper = UndistortRemap(image_size, K, D, rH)
roi_remapper = RoiUndistortRemap(roi_x, roi_y, roi_w, roi_h, remapper)
rmap_x = np.zeros(shape=(roi_h, roi_w), dtype=np.float32)
rmap_y = np.zeros(shape=(roi_h, roi_w), dtype=np.float32)

for x in np.arange(0, roi_w):
    for y in np.arange(0, roi_h):
        rmap_x[y, x] = roi_remapper.x(x, y)
        rmap_y[y, x] = roi_remapper.y(x, y)

In [14]:
# class UndistortRemap:
#     def __init__(self, image_size, K, D, rH):
#         self.image_size = image_size
#         self.K = K
#         self.D = D
#         self.rH = rH
        
#     def r2(self, x, y):
#         return (x**2 + y**2)
    
#     def coef(self, x, y):
#         r2_ = self.r2(x, y)
#         r4_ = r2_ ** 2
#         r6_ = r4_ * r2_
#         return (1 + self.D[0] * r2_ + self.D[1] * r4_ + self.D[4] * r6_)
        
#     def normalize(self, x, y):
#         return ((x - self.K[0][2]) / self.K[0][0], (y - self.K[1][2]) / self.K[1][1])
    
#     def unnormalize(self, x, y):
#         return (x * self.K[0][0] + self.K[0][2], y * self.K[1][1] + self.K[1][2])
    
#     def proj(self, x, y):
#         pt_homo = np.linalg.inv(self.rH).dot(np.array([x, y, 1], dtype=np.float32).T)
#         return (pt_homo[0] / pt_homo[2], pt_homo[1] / pt_homo[2])

#     def undistort(self, x, y):
#         x_norm, y_norm = self.normalize(x, y)
#         xx, yy = self.proj(x_norm, y_norm)
#         x_c = xx * self.coef(xx, yy)
#         y_c = yy * self.coef(xx, yy)
#         xx_1, yy_1, a = np.array([[self.rH[2][2], 0, -self.rH[0][2]], [0, self.rH[2][2], -self.rH[1][2]], [0, 0, 1]], dtype=np.float32).dot(self.rH).dot(np.array([x_c, y_c, 1], dtype=np.float32))
#         x_res, y_res = self.unnormalize(xx_1, yy_1)
#         return (x_res, y_res)
    
#     def roi_undistort(self, x, y, roi_x, roi_y, roi_w, roi_h):
#         if x >= roi_x and x <= roi_x+roi_w and y >= roi_y and y <= roi_y+roi_h:
#             x_res, y_res = self.undistort(x, y)
#         else:
#             x_res = x
#             y_res = y
#         return x_res, y_res

In [15]:
# remapper = UndistortRemap(image_size, K, D, rH)

# rmap_x = np.zeros(shape=(image_size[1], image_size[0]), dtype=np.float32)
# rmap_y = np.zeros(shape=(image_size[1], image_size[0]), dtype=np.float32)

# width = image_size[0]
# height = image_size[1]

# for x in np.arange(0, width):
#     for y in np.arange(0, height):
#         x_res, y_res = remapper.roi_undistort(x, y, roi_x, roi_y, roi_w, roi_h)
#         if x_res >= 0 and x_res < width and y_res >= 0 and y_res < height:
#             rmap_x[y, x] = x_res
#             rmap_y[y, x] = y_res

In [16]:
undistort_roi = cv2.remap(i, rmap_x, rmap_y, interpolation=cv2.INTER_LINEAR)

cv2.imwrite("roi_undistort.png", undistort_roi)

True

![undistort-roi](./roi_undistort.png)

## Задача 3
Вычислить карту пересчета преобразования birdeye после выполнения undistort. Сравнить с результатами birdeye из лекции

In [24]:
class BirdeyeRemap:
    def __init__(self, H):
        self.H = H
    
    def proj(self, x, y):
        pt_homo = self.H @ np.array([x, y, 1], dtype=np.float32).T
        return (pt_homo[0] / pt_homo[2], pt_homo[1] / pt_homo[2])
    
    def x(self, x, y):
        return self.proj(x, y)[0]
    
    def y(self, x, y):
        return self.proj(x, y)[1]

In [31]:
# матрица камеры проекции: такая же камер, но висящая над сценой и смотрящая вниз
r1_beye = -0.5 * np.pi
r2_beye = 0.0
r3_beye = 0.0

t1_beye = 0.0
t2_beye = -10 # висеть будем на высоте 10 м
t3_beye = 10 # и на 10 м вперед относительно начала координат

# матрица поворота
R_beye = np.linalg.inv(cv2.Rodrigues(np.array([r1_beye, r2_beye, r3_beye], dtype=np.float32))[0])

t_beye = np.array([[t1_beye], [t2_beye], [t3_beye]], dtype=np.float32)
RT_beye = np.hstack((R_beye, -R_beye@t_beye))
print(f"R|t = \n{RT_beye}")

# матрица камеры наблюдения
P_beye = K @ RT_beye
print(f"P = \n{P_beye}")

# уравнение плоскости проекции
plane = np.array([0,1,0,0], dtype=np.float32) # Y = 0

# применим формулы для вычисления гомографии с предыдущей лекции
P_plane = np.vstack((P_beye, plane))
H = P @ np.linalg.inv(P_plane)[:, :3]

print(H)

R|t = 
[[ 1.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00]
 [-0.000000e+00 -4.371139e-08 -1.000000e+00  1.000000e+01]
 [ 0.000000e+00  1.000000e+00 -4.371139e-08  1.000000e+01]]
P = 
[[ 9.4337000e+02  9.9038000e+02 -4.3290886e-05  9.9037998e+03]
 [ 0.0000000e+00  4.8766998e+02 -9.4337000e+02  1.4310400e+04]
 [ 0.0000000e+00  1.0000000e+00 -4.3711388e-08  1.0000000e+01]]
[[ 9.5320189e-01 -8.7875682e-01  3.8827081e+02]
 [-2.4479324e-02  1.7686374e-01  2.2081152e+02]
 [-4.6287762e-05 -8.5292436e-04  1.3034890e+00]]


In [26]:
img_homo = np.zeros_like(my_undistort)
remapper = BirdeyeRemap(H)

width = image_size[0]
height = image_size[1]

for x in np.arange(0, width):
    for y in np.arange(0, height):
        xx, yy = remapper.proj(x,y)
        if xx >= 0 and xx < width and yy >= 0 and yy < height:
            img_homo[y,x,:] = bilinear_kernel(my_undistort, xx, yy)

cv2.imwrite("beye.png", img_homo)

True

Преобразование birdeye после undistort
![beye](./beye.png)

In [32]:
img_homo = np.zeros_like(i)
remapper = BirdeyeRemap(H)

width = image_size[0]
height = image_size[1]

for x in np.arange(0, width):
    for y in np.arange(0, height):
        xx, yy = remapper.proj(x,y)
        if xx >= 0 and xx < width and yy >= 0 and yy < height:
            img_homo[y,x,:] = bilinear_kernel(i, xx, yy)

cv2.imwrite("beye_i.png", img_homo)

True

Преобразование birdeye до undistort
![beye_i](./beye_i.png)